# Bluebook for Bulldozers — End-to-End Regression (RMSLE)

Run this notebook **top-to-bottom**. It is written to prevent:
- *File not found* (auto-detects CSV location/name)
- *DTypePromotionError* (drops datetime after extracting date parts)
- *KeyError on dropped columns* (recomputes columns + rebuilds pipeline after transforms)


In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error, r2_score, make_scorer
import joblib

from src.file_utils import find_kaggle_csv
from src.preprocess import add_dateparts

MODELS_DIR = Path("models"); MODELS_DIR.mkdir(exist_ok=True)
REPORTS_DIR = Path("reports"); REPORTS_DIR.mkdir(exist_ok=True)


## 1) Data ingestion + date parsing + sorting

In [ ]:
train_path = find_kaggle_csv("TrainAndValid")
print("Using:", train_path)

df = pd.read_csv(train_path, low_memory=False, parse_dates=["saledate"])
df = df.sort_values("saledate").reset_index(drop=True)

df.shape, df["saledate"].min(), df["saledate"].max()


## 2) Time-series split (no random shuffle)

In [ ]:
split_date = pd.to_datetime("2012-01-01")
train_df = df[df["saledate"] < split_date].copy()
valid_df = df[df["saledate"] >= split_date].copy()
train_df.shape, valid_df.shape


## 3) Feature engineering from `saledate` (drop raw datetime)

In [ ]:
train_df = add_dateparts(train_df, "saledate", drop=True)
valid_df = add_dateparts(valid_df, "saledate", drop=True)

assert "saledate" not in train_df.columns
assert "saledate" not in valid_df.columns


## 4) Initial inspection (dtypes, missing values, unique values)

In [ ]:
print("Train FE shape:", train_df.shape)
display(train_df.dtypes)

missing = train_df.isna().sum().sort_values(ascending=False)
display(missing.head(25))
print("Columns with missing:", int((missing > 0).sum()))

cols_to_check = ["SalePrice", "YearMade", "MachineID", "ModelID", "state", "ProductSize", "UsageBand"]
cols_to_check = [c for c in cols_to_check if c in train_df.columns]
for c in cols_to_check:
    print(f"\n--- {c} ---")
    print("dtype:", train_df[c].dtype, "nunique:", train_df[c].nunique(dropna=False))
    display(train_df[c].value_counts(dropna=False).head(10))


## 5) Build X/y and preprocess (categories, missing values, encoding)

In [ ]:
target = "SalePrice"

X_train = train_df.drop(columns=[target])
y_train = train_df[target].values
X_valid = valid_df.drop(columns=[target])
y_valid = valid_df[target].values

for c in X_train.columns:
    if X_train[c].dtype == "object":
        X_train[c] = X_train[c].astype("category")
for c in X_valid.columns:
    if X_valid[c].dtype == "object":
        X_valid[c] = X_valid[c].astype("category")

dt_cols = list(X_train.select_dtypes(include=["datetime64[ns]", "datetime64"]).columns)
print("Datetime cols in X_train:", dt_cols)
if dt_cols:
    X_train = X_train.drop(columns=dt_cols)
    X_valid = X_valid.drop(columns=dt_cols, errors="ignore")

X_valid = X_valid.reindex(columns=X_train.columns)

cat_cols = [c for c in X_train.columns if str(X_train[c].dtype) in ("category", "object")]
num_cols = [c for c in X_train.columns if c not in cat_cols]
len(num_cols), len(cat_cols)


## 6) Modeling + evaluation (baseline)

In [ ]:
numeric = Pipeline([("imputer", SimpleImputer(strategy="median"))])
categorical = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
])

preprocess = ColumnTransformer([
    ("num", numeric, num_cols),
    ("cat", categorical, cat_cols),
], remainder="drop", sparse_threshold=0.0)

model = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1,
    max_features="sqrt",
)

pipe = Pipeline([("preprocess", preprocess), ("model", model)])

def evaluate(y_true, y_pred):
    y_pred_clip = np.maximum(0, y_pred)
    return {
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "rmsle": float(np.sqrt(mean_squared_log_error(y_true, np.maximum(0, y_pred_clip)))),
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "r2": float(r2_score(y_true, y_pred)),
    }

pipe.fit(X_train, y_train)
pred = pipe.predict(X_valid)
baseline_metrics = evaluate(y_valid, pred)
baseline_metrics


## 7) Hyperparameter tuning (RandomizedSearchCV)

In [ ]:
def neg_rmsle(y_true, y_pred):
    y_pred = np.maximum(0, y_pred)
    return -float(np.sqrt(mean_squared_log_error(y_true, y_pred)))

scorer = make_scorer(neg_rmsle, greater_is_better=True)

param_dist = {
    "model__n_estimators": [200, 400, 700],
    "model__max_depth": [None, 10, 20, 30],
    "model__max_features": ["sqrt", "log2", 0.5],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4],
}

search = RandomizedSearchCV(
    pipe,
    param_distributions=param_dist,
    n_iter=20,
    scoring=scorer,
    cv=3,
    random_state=42,
    n_jobs=-1,
    verbose=1,
)
search.fit(X_train, y_train)

best_model = search.best_estimator_
best_params = search.best_params_

tuned_pred = best_model.predict(X_valid)
tuned_metrics = evaluate(y_valid, tuned_pred)

baseline_metrics, tuned_metrics, best_params


## 8) Save model + metrics

In [ ]:
joblib.dump(best_model, MODELS_DIR/"model.joblib")

metrics_out = {
    "baseline": baseline_metrics,
    "tuned": tuned_metrics,
    "best_params": best_params,
    "split_date": "2012-01-01",
    "train_path_used": str(train_path),
}
(MODELS_DIR/"metrics.json").write_text(json.dumps(metrics_out, indent=2))

metrics_out


## 9) Test predictions + custom prediction

In [ ]:
test_path = find_kaggle_csv("Test")
print("Using:", test_path)

test_df = pd.read_csv(test_path, low_memory=False, parse_dates=["saledate"])
test_df = test_df.sort_values("saledate").reset_index(drop=True)
test_df = add_dateparts(test_df, "saledate", drop=True)

for c in test_df.columns:
    if test_df[c].dtype == "object":
        test_df[c] = test_df[c].astype("category")

test_pred = best_model.predict(test_df)
test_pred[:5], len(test_pred)


In [ ]:
sample = {
    "saledate": "2012-11-19",
    "MachineID": 999999,
    "ModelID": 4605,
    "fiBaseModel": 555,
    "YearMade": 2005,
    "ProductSize": "Medium",
    "state": "Florida",
    "UsageBand": "High",
}
sample_df = pd.DataFrame([sample])
sample_df = add_dateparts(sample_df, "saledate", drop=True)
for c in sample_df.columns:
    if sample_df[c].dtype == "object":
        sample_df[c] = sample_df[c].astype("category")

float(best_model.predict(sample_df)[0])


## 10) Feature importance

In [ ]:
rf = best_model.named_steps["model"]
importances = rf.feature_importances_
feature_names = num_cols + cat_cols

fi = pd.DataFrame({"feature": feature_names, "importance": importances}).sort_values("importance", ascending=False)
fi.to_csv(REPORTS_DIR/"feature_importance.csv", index=False)

top = fi.head(20)[::-1]
plt.figure(figsize=(8, 10))
plt.barh(top["feature"], top["importance"])
plt.title("Top 20 Feature Importances (RandomForest)")
plt.tight_layout()
plt.savefig(REPORTS_DIR/"feature_importance_top20.png", dpi=200)
plt.show()

fi.head(10)
